In [ ]:
!pip install -U -q pydrive google-colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

# Authenticate and mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
from tensorflow.keras.callbacks import EarlyStopping

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results2"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)
print("Size of DataFrame after dropping null values:", df.shape)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            if vec.shape[0] == expected_dim:
                return vec
            elif vec.shape[0] > expected_dim:
                return vec[:expected_dim]  # Truncate to expected size
            else:
                return np.pad(vec, (0, expected_dim - vec.shape[0]))  # Pad with zeros
    except:
        return np.zeros(expected_dim, dtype=np.float32)  # Default to zero vector
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
word2vec_dim = 300
fasttext_dim = 300
sentence_embedding_dim = 300
sinbert_dim = 768  # Larger than others

# Apply parsing with dimension corrections
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, word2vec_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, fasttext_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, sentence_embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, sinbert_dim))  # Keep 768D

# Ensure all feature vectors have the same final dimension
final_dim = 768  # Choose larger or normalize to 300
for feature in ['word2vec_vector', 'fasttext_vector', 'sentence_embedding']:
    df[feature] = df[feature].map(lambda x: np.pad(x, (0, final_dim - x.shape[0])) if x.shape[0] < final_dim else x[:final_dim])

# Stack feature vectors correctly
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)

# Flatten X for MLP: (samples, final_dim * 4)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Function to implement early stopping and class weights
def build_mlp(input_shape, dropout_rate=0.2, dense_units=128):
    model = Sequential([
        Input(shape=input_shape),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(dense_units // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape, dropout_rate=0.2, lstm_units=64):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units // 2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape, dropout_rate=0.2, filters=64):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters, kernel_size=3, activation='relu'),
        Dropout(dropout_rate),
        GlobalMaxPooling1D(),
        Dense(filters // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Apply class weights (use if there is class imbalance)
class_weights = {0: 1, 1: 1.5}  # Increase weight for the minority class

# Evaluate the model
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix.tolist()
    }

# Train and evaluate models with hyperparameter tuning and regularization
results = {}

# Model hyperparameters for tuning
params = {
    'MLP': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'dense_units': [64, 128]
    },
    'LSTM': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'lstm_units': [64, 128],
    },
    'CNN': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'filters': [64, 128],
    }
}

# Train and evaluate each model
for name in ['MLP', 'LSTM', 'CNN']:
    print(f"Training {name}...")

    # Hyperparameter grid search (manually tuned here)
    for epochs in params[name]['epochs']:
        for batch_size in params[name]['batch_size']:
            if name == 'MLP':
                for dropout in params[name]['dropout']:
                    for dense_units in params[name]['dense_units']:
                        model_instance = build_mlp((X_train_flattened.shape[1],), dropout, dense_units)
                        model_instance.fit(X_train_flattened, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test_flattened, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test_flattened) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'LSTM':
                for dropout in params[name]['dropout']:
                    for lstm_units in params[name]['lstm_units']:
                        model_instance = build_lstm((X_train.shape[1], X_train.shape[2]), dropout, lstm_units)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'CNN':
                for dropout in params[name]['dropout']:
                    for filters in params[name]['filters']:
                        model_instance = build_cnn((X_train.shape[1], X_train.shape[2]), dropout, filters)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)

# Save results to JSON
results_path = os.path.join(save_path, "model_results.json")
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)

print("Experiment completed. Results saved.")


Size of DataFrame after dropping null values: (4958, 8)
Training MLP...
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7070 - loss: 0.6495 - val_accuracy: 0.7994 - val_loss: 0.4507
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8067 - loss: 0.5036 - val_accuracy: 0.8075 - val_loss: 0.4578
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8089 - loss: 0.4938 - val_accuracy: 0.8135 - val_loss: 0.4263
Epoch 4/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8163 - loss: 0.4820 - val_accuracy: 0.8075 - val_loss: 0.4538
Epoch 5/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8235 - loss: 0.4737 - val_accuracy: 0.8105 - val_loss: 0.4440
Epoch 6/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8373 - loss: 0.4499 - val_accuracy: 0.8004 - val_loss: 0.4636
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7215 - loss: 0.6177 - val_accuracy: 0.7087 - val_loss: 

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
from tensorflow.keras.callbacks import EarlyStopping

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results5"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)
print("Size of DataFrame after dropping null values:", df.shape)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            if vec.shape[0] == expected_dim:
                return vec
            elif vec.shape[0] > expected_dim:
                return vec[:expected_dim]  # Truncate to expected size
            else:
                return np.pad(vec, (0, expected_dim - vec.shape[0]))  # Pad with zeros
    except:
        return np.zeros(expected_dim, dtype=np.float32)  # Default to zero vector
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
word2vec_dim = 300
fasttext_dim = 300
sentence_embedding_dim = 300
sinbert_dim = 768  # Larger than others

# Apply parsing with dimension corrections
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, word2vec_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, fasttext_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, sentence_embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, sinbert_dim))  # Keep 768D

# Ensure all feature vectors have the same final dimension
final_dim = 768  # Choose larger or normalize to 300
for feature in ['word2vec_vector', 'fasttext_vector', 'sentence_embedding']:
    df[feature] = df[feature].map(lambda x: np.pad(x, (0, final_dim - x.shape[0])) if x.shape[0] < final_dim else x[:final_dim])

# Stack feature vectors correctly
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)

# Flatten X for MLP: (samples, final_dim * 4)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Function to implement early stopping and class weights
def build_mlp(input_shape, dropout_rate, dense_units):
    model = Sequential([
        Input(shape=input_shape),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(dense_units // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape, dropout_rate, lstm_units):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units // 2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape, dropout_rate, filters):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters, kernel_size=3, activation='relu'),
        Dropout(dropout_rate),
        GlobalMaxPooling1D(),
        Dense(filters // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Apply class weights (use if there is class imbalance)
class_weights = {0: 1, 1: 1.5}  # Increase weight for the minority class

# Evaluate the model
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix.tolist()
    }

# Train and evaluate models with hyperparameter tuning and regularization
results = {}

# Model hyperparameters for tuning
params = {
    'MLP': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'dense_units': [64, 128]
    },
    'LSTM': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'lstm_units': [64, 128],
    },
    'CNN': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'filters': [64, 128],
    }
}

# Train and evaluate each model
for name in ['MLP', 'LSTM', 'CNN']:
    print(f"Training {name}...")

    # Hyperparameter grid search (manually tuned here)
    for epochs in params[name]['epochs']:
        for batch_size in params[name]['batch_size']:
            if name == 'MLP':
                for dropout in params[name]['dropout']:
                    for dense_units in params[name]['dense_units']:
                        model_instance = build_mlp((X_train_flattened.shape[1],), dropout, dense_units)
                        model_instance.fit(X_train_flattened, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test_flattened, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test_flattened) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'LSTM':
                for dropout in params[name]['dropout']:
                    for lstm_units in params[name]['lstm_units']:
                        model_instance = build_lstm((X_train.shape[1], X_train.shape[2]), dropout, lstm_units)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'CNN':
                for dropout in params[name]['dropout']:
                    for filters in params[name]['filters']:
                        model_instance = build_cnn((X_train.shape[1], X_train.shape[2]), dropout, filters)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)

# Save results to JSON
results_path = os.path.join(save_path, "model_results.json")
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)

print("Experiment completed. Results saved.")

Size of DataFrame after dropping null values: (4958, 8)
Training MLP...
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7140 - loss: 0.6325 - val_accuracy: 0.8095 - val_loss: 0.4467
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8067 - loss: 0.5083 - val_accuracy: 0.7873 - val_loss: 0.4730
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8233 - loss: 0.4815 - val_accuracy: 0.7913 - val_loss: 0.4774
Epoch 4/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8133 - loss: 0.4765 - val_accuracy: 0.7792 - val_loss: 0.4890
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.7075 - loss: 0.6428 - val_accuracy: 0.7903 - val_loss: 0.4430
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8010 - loss: 0.5273 - val_accuracy: 0.8034 - val_loss: 0.4346
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8219 - loss: 0.4804 - val_accuracy: 0.7994 - val_loss

Size of DataFrame after dropping null values: (4958, 8)
Training MLP...
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.7410 - loss: 0.6352 - val_accuracy: 0.8034 - val_loss: 0.4331
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7975 - loss: 0.5205 - val_accuracy: 0.8054 - val_loss: 0.4397
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8096 - loss: 0.4943 - val_accuracy: 0.8044 - val_loss: 0.4558
Epoch 4/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8306 - loss: 0.4660 - val_accuracy: 0.8054 - val_loss: 0.4368
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7369 - loss: 0.6140 - val_accuracy: 0.7712 - val_loss: 0.4973
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7986 - loss: 0.5121 - val_accuracy: 0.7712 - val_loss: 0.4900
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8150 - loss: 0.4867 - val_accuracy: 0.8054 - val_los

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, Input, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
from tensorflow.keras.callbacks import EarlyStopping

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results4"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)
print("Size of DataFrame after dropping null values:", df.shape)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            if vec.shape[0] == expected_dim:
                return vec
            elif vec.shape[0] > expected_dim:
                return vec[:expected_dim]  # Truncate to expected size
            else:
                return np.pad(vec, (0, expected_dim - vec.shape[0]))  # Pad with zeros
    except:
        return np.zeros(expected_dim, dtype=np.float32)  # Default to zero vector
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
word2vec_dim = 300
fasttext_dim = 300
sentence_embedding_dim = 300
sinbert_dim = 768  # Larger than others

# Apply parsing with dimension corrections
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, word2vec_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, fasttext_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, sentence_embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, sinbert_dim))  # Keep 768D

# Ensure all feature vectors have the same final dimension
final_dim = 768  # Normalize to 768
for feature in ['word2vec_vector', 'fasttext_vector', 'sentence_embedding']:
    df[feature] = df[feature].map(lambda x: np.pad(x, (0, final_dim - x.shape[0])) if x.shape[0] < final_dim else x[:final_dim])

# Stack feature vectors correctly
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)

# Reshape X for BiLSTM (samples, time_steps, features)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define BiLSTM Model
def build_bilstm(input_shape, lstm_units=128, dropout_rate=0.3):
    model = Sequential([
        Input(shape=input_shape),
        Bidirectional(LSTM(lstm_units, return_sequences=True)),
        Dropout(dropout_rate),
        Bidirectional(LSTM(lstm_units // 2)),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train BiLSTM model
bilstm_model = build_bilstm((X_train.shape[1], X_train.shape[2]))
bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test),
                 callbacks=[early_stopping], verbose=1)

# Predict and evaluate
y_pred = (bilstm_model.predict(X_test) > 0.5).astype(int)

# Define evaluation function
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix.tolist()
    }

# Save results
results = evaluate_model(y_test, y_pred)
results_path = os.path.join(save_path, "bilstm_results.json")
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)

# Save model
bilstm_model.save(os.path.join(save_path, "bilstm_model.h5"))

print("Experiment completed. Results saved.")


Size of DataFrame after dropping null values: (4958, 8)
Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 480s 4s/step - accuracy: 0.5806 - loss: 0.6640 - val_accuracy: 0.6425 - val_loss: 0.6305
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 497s 4s/step - accuracy: 0.6428 - loss: 0.6383 - val_accuracy: 0.6499 - val_loss: 0.6151
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 503s 4s/step - accuracy: 0.6587 - loss: 0.6138 - val_accuracy: 0.6680 - val_loss: 0.6024
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 435s 4s/step - accuracy: 0.6762 - loss: 0.5992 - val_accuracy: 0.6687 - val_loss: 0.6001
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 475s 4s/step - accuracy: 0.6773 - loss: 0.6029 - val_accuracy: 0.7050 - val_loss: 0.5658
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 503s 4s/step - accuracy: 0.7061 - loss: 0.5531 - val_accuracy: 0.7325 - val_loss: 0.5487
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 500s 4s/step - accuracy: 0.7296 - loss: 0.5288 - val_accuracy: 0.7466 - val_loss: 0.5234
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 4

Experiment completed. Results saved.


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
from tensorflow.keras.callbacks import EarlyStopping

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results2"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)
print("Size of DataFrame after dropping null values:", df.shape)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            if vec.shape[0] == expected_dim:
                return vec
            elif vec.shape[0] > expected_dim:
                return vec[:expected_dim]  # Truncate to expected size
            else:
                return np.pad(vec, (0, expected_dim - vec.shape[0]))  # Pad with zeros
    except:
        return np.zeros(expected_dim, dtype=np.float32)  # Default to zero vector
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
word2vec_dim = 300
fasttext_dim = 300
sentence_embedding_dim = 300
sinbert_dim = 768  # Larger than others

# Apply parsing with dimension corrections
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, word2vec_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, fasttext_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, sentence_embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, sinbert_dim))  # Keep 768D

# Ensure all feature vectors have the same final dimension
final_dim = 768  # Choose larger or normalize to 300
for feature in ['word2vec_vector', 'fasttext_vector', 'sentence_embedding']:
    df[feature] = df[feature].map(lambda x: np.pad(x, (0, final_dim - x.shape[0])) if x.shape[0] < final_dim else x[:final_dim])

# Stack feature vectors correctly
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)

# Flatten X for MLP: (samples, final_dim * 4)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Function to implement early stopping and class weights
def build_mlp(input_shape, dropout_rate=0.2, dense_units=128):
    model = Sequential([
        Input(shape=input_shape),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(dense_units // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape, dropout_rate=0.2, lstm_units=64):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units // 2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape, dropout_rate=0.2, filters=64):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters, kernel_size=3, activation='relu'),
        Dropout(dropout_rate),
        GlobalMaxPooling1D(),
        Dense(filters // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Apply class weights (use if there is class imbalance)
class_weights = {0: 1, 1: 1.5}  # Increase weight for the minority class

# Evaluate the model
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix.tolist()
    }

# Train and evaluate models with hyperparameter tuning and regularization
results = {}

# Model hyperparameters for tuning
params = {
    'MLP': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'dense_units': [64, 128]
    },
    'LSTM': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'lstm_units': [64, 128],
    },
    'CNN': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'filters': [64, 128],
    }
}

# Train and evaluate each model
for name in ['MLP', 'LSTM', 'CNN']:
    print(f"Training {name}...")

    # Hyperparameter grid search (manually tuned here)
    for epochs in params[name]['epochs']:
        for batch_size in params[name]['batch_size']:
            if name == 'MLP':
                for dropout in params[name]['dropout']:
                    for dense_units in params[name]['dense_units']:
                        model_instance = build_mlp((X_train_flattened.shape[1],), dropout, dense_units)
                        model_instance.fit(X_train_flattened, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test_flattened, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test_flattened) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'LSTM':
                for dropout in params[name]['dropout']:
                    for lstm_units in params[name]['lstm_units']:
                        model_instance = build_lstm((X_train.shape[1], X_train.shape[2]), dropout, lstm_units)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'CNN':
                for dropout in params[name]['dropout']:
                    for filters in params[name]['filters']:
                        model_instance = build_cnn((X_train.shape[1], X_train.shape[2]), dropout, filters)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)

# Save results to JSON
results_path = os.path.join(save_path, "model_results.json")
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)

print("Experiment completed. Results saved.")